# Example of DOV search methods for hydrogeological interpretations

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_hydrogeologische_stratigrafie.ipynb)

## Use cases:
* Select interpretations in a bbox
* Select interpretations in a bbox with selected properties
* Select interpretations in a municipality
* Get interpretations using info from wfs fields, not available in the standard output dataframe

In [1]:
%matplotlib inline
import os, sys
import inspect

In [2]:
import pydov

## Get information about code base

In [3]:
from pydov.search.interpretaties import HydrogeologischeStratigrafieSearch
ip_hydrogeo = HydrogeologischeStratigrafieSearch()

# information about the HydrogeologischeStratigrafie type (In Dutch):
print(ip_hydrogeo.get_description())

De hydrostratigrafie geeft, op basis van de (gecodeerde) lithologie, een indeling weer naar de al dan niet watervoerende eigenschappen van een bepaald beschreven diepte-interval. Deze interpretatie respecteert de lithostratigrafie van het Tertiair, maar deelt deze anders in. De hiervoor gebruikte standaard is de Hydrogeologische Codering van de Ondergrond van Vlaanderen (HCOV). Deze kan beschouwd worden als de officiele hydrogeologische codering voor het Vlaams Gewest.


In [4]:
# information about the available fields for a HydrogeologischeStratigrafie object
fields = ip_hydrogeo.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
pkey_boring
x
y
start_interpretatie_mtaw
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
eerste_invoer
geom
diepte_laag_van
diepte_laag_tot
aquifer


In [5]:
# print information for a certain field
fields['aquifer']

{'name': 'aquifer', 'type': 'string', 'list': False, 'definition': 'Code van de watervoerende laag waarin de laag Hydrogeologische stratigrafie zich bevindt (HCOVv1)', 'notnull': False, 'query': False, 'cost': 10, 'codelist': <pydov.util.codelists.AbstractCodeList: <pydov.util.codelists.CodeListItem: code: 0000, label: 0000, definition: Onbekend>, <pydov.util.codelists.CodeListItem: code: 0100, label: 0100, definition: Quartaire aquifersystemen>, <pydov.util.codelists.CodeListItem: code: 0110, label: 0110, definition: Ophogingen>, <pydov.util.codelists.CodeListItem: code: 0120, label: 0120, definition: Duinen>, <pydov.util.codelists.CodeListItem: code: 0130, label: 0130, definition: Polderafzettingen>, <pydov.util.codelists.CodeListItem: code: 0131, label: 0131, definition: Kleiige polderafzettingen van de kustvlakte>, <pydov.util.codelists.CodeListItem: code: 0132, label: 0132, definition: Kleiige polderafzettingen van het Meetjesland>, <pydov.util.codelists.CodeListItem: code: 0133, label: 0133, definition: Kleiige polderafzettingen van Waasland-Antwerpen>, <pydov.util.codelists.CodeListItem: code: 0134, label: 0134, definition: Zandige kreekruggen>, <pydov.util.codelists.CodeListItem: code: 0135, label: 0135, definition: Veen-kleiige poelgronden>, <pydov.util.codelists.CodeListItem: code: 0140, label: 0140, definition: Alluviale deklagen>, <pydov.util.codelists.CodeListItem: code: 0150, label: 0150, definition: Deklagen>, <pydov.util.codelists.CodeListItem: code: 0151, label: 0151, definition: Zandige deklagen>, <pydov.util.codelists.CodeListItem: code: 0152, label: 0152, definition: Zand-lemige deklagen>, <pydov.util.codelists.CodeListItem: code: 0153, label: 0153, definition: Lemige deklagen>, <pydov.util.codelists.CodeListItem: code: 0154, label: 0154, definition: Kleiige deklagen>, <pydov.util.codelists.CodeListItem: code: 0160, label: 0160, definition: Pleistocene afzettingen>, <pydov.util.codelists.CodeListItem: code: 0161, label: 0161, definition: Pleistoceen van de kustvlakte>, <pydov.util.codelists.CodeListItem: code: 0162, label: 0162, definition: Pleistoceen van de Vlaamse Vallei>, <pydov.util.codelists.CodeListItem: code: 0163, label: 0163, definition: Pleistoceen van de riviervalleien>, <pydov.util.codelists.CodeListItem: code: 0170, label: 0170, definition: Maas- en Rijnafzettingen>, <pydov.util.codelists.CodeListItem: code: 0171, label: 0171, definition: Afzettingen Hoofdterras>, <pydov.util.codelists.CodeListItem: code: 0172, label: 0172, definition: Afzettingen Tussenterassen>, <pydov.util.codelists.CodeListItem: code: 0173, label: 0173, definition: Afzettingen Maasvlakte>, <pydov.util.codelists.CodeListItem: code: 0200, label: 0200, definition: Kempens Aquifersysteem>, <pydov.util.codelists.CodeListItem: code: 0210, label: 0210, definition: Kiezeloolietformatie ten noorden van Feldbiss>, <pydov.util.codelists.CodeListItem: code: 0211, label: 0211, definition: Zandige eenheid boven de Brunssum I-klei>, <pydov.util.codelists.CodeListItem: code: 0212, label: 0212, definition: Brunssum I-Klei>, <pydov.util.codelists.CodeListItem: code: 0213, label: 0213, definition: Zand van Pey>, <pydov.util.codelists.CodeListItem: code: 0214, label: 0214, definition: Brunssum II-klei>, <pydov.util.codelists.CodeListItem: code: 0215, label: 0215, definition: Zand van Waubach>, <pydov.util.codelists.CodeListItem: code: 0220, label: 0220, definition: Klei-zand-complex van de Kempen>, <pydov.util.codelists.CodeListItem: code: 0221, label: 0221, definition: Klei van Turnhout>, <pydov.util.codelists.CodeListItem: code: 0222, label: 0222, definition: Zand van Beerse>, <pydov.util.codelists.CodeListItem: code: 0223, label: 0223, definition: Klei van Rijkevorsel>, <pydov.util.codelists.CodeListItem: code: 0230, label: 0230, definition: Pleistoceen en Plioceen aquifer>, <pydov.util.codelists.CodeListItem: code: 0231, label: 0231, definition: Zanden van Brasschaat en/of Merksplas>, <pydov.util.codelists.CodeListItem: code: 0232, label:

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

In [6]:
# if an attribute can have several values, these are listed under 'codelist', e.g. for 'Type_proef':
fields['Type_proef']

{'name': 'Type_proef', 'definition': "Het type van de proef waarvan de hydrogeologische stratigrafie een interpretatie is. Bij een hydrogeologische stratigrafie zal dit altijd 'Boring' zijn.", 'type': 'string', 'list': False, 'notnull': False, 'query': True, 'cost': 1, 'codelist': <pydov.util.codelists.AbstractCodeList: <pydov.util.codelists.CodeListItem: code: Boring, label: Boring, definition: De hydrogeologische stratigrafie is een interpretatie van een boring.>>}

Alternatively, you can list all the fields and their details by inspecting the `get_fields()` output or the search instance itself in a notebook:

In [7]:
ip_hydrogeo

## Try-out of use cases

### Select interpretations in a bbox

In [22]:
from pydov.util.location import Within, Box

# Get all borehole data in a bounding box (llx, lly, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = ip_hydrogeo.search(location=Within(Box(153145, 206930, 153150, 206935)))
df.head()

[000/001] .
[000/001] c


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,aquifer
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,0.0,2.5,0152
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,2.5,7.0,0254


You can also add the `aquifer` definitions:

In [23]:
df['aquifer_definition'] = df['aquifer'].map(ip_hydrogeo.get_fields()['aquifer']['codelist'].get_definition)
df

,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,aquifer,aquifer_definition
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,0.0,2.5,0152,Zand-lemige deklagen
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,2.5,7.0,0254,Zanden van Berchem en/of Voort


### Select interpretations in a bbox with selected properties

In [24]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes2'], 
                                           inspect.isclass) 
           if 'Property' in i]
print(methods)

['PropertyIsBetween', 'PropertyIsEqualTo', 'PropertyIsGreaterThan', 'PropertyIsGreaterThanOrEqualTo', 'PropertyIsLessThan', 'PropertyIsLessThanOrEqualTo', 'PropertyIsLike', 'PropertyIsNotEqualTo', 'PropertyIsNull', 'SortProperty']


The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [25]:
# Get deep boreholes in a bounding box 
from owslib.fes2 import PropertyIsGreaterThanOrEqualTo
# the propertyname can be any of the fields of the hydrogeological interpretations object that belong to the wfs source
# the literal is always a string, no matter what its definition is in the boring object (string, float...)
query = PropertyIsGreaterThanOrEqualTo(
        propertyname='betrouwbaarheid_interpretatie', literal='goed')
df = ip_hydrogeo.search(location=Within(Box(153145, 206930, 153150, 206935)),
                        query=query
                       )
df.head()

[000/001] .
[000/001] c


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,aquifer
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,0.0,2.5,0152
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,goed,153147.0,206931.0,14.12,2.5,7.0,0254


### Select interpretations in a municipality

In [26]:
from owslib.fes2 import PropertyIsEqualTo

query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Aartselaar')
df = ip_hydrogeo.search(query=query)
df.head()

[000/001] .
[000/006] cccccc


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,aquifer
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1950...,goed,151457.00,201527.00,21.00,0.0,49.0,0300
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1950...,goed,151457.00,201527.00,21.00,49.0,59.0,0400
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2003...,goed,151994.14,203642.73,11.75,0.0,2.0,0100
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2003...,goed,151994.14,203642.73,11.75,2.0,4.0,0300
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1894...,goed,150678.00,200279.00,20.00,0.0,2.7,0160


### Get interpretations using info from wfs fields, not available in the standard output dataframe

To keep the dataframe size acceptable, not all wfs fields are included in the standard output. However, one can use this information to select interpretations as illustrated below.
The available wfs fields can be viewed in the object catalog of the 'Hydrogeologische stratigrafie' wfs service, on-line [2018 05 16]: https://www.dov.vlaanderen.be/geonetwork/apps/tabsearch/index.html?hl=dut.
For example, make a selection of the interpretations in the municipality of Antwerp that were derived from cpt data

In [27]:
from owslib.fes2 import And
from owslib.fes2 import PropertyIsEqualTo
query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             PropertyIsEqualTo(propertyname='Type_proef', 
                               literal='Boring')]
            )
df = ip_hydrogeo.search(query=query, )
df.head()

[000/001] .
[000/130] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/130] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/130] cccccccccccccccccccccccccccccc


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,aquifer
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1965...,goed,144298.7,228180.2,2.00,0.0,1.00,0133
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1965...,goed,144298.7,228180.2,2.00,1.0,4.00,0134
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1965...,goed,144298.7,228180.2,2.00,4.0,6.00,0135
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1965...,goed,144298.7,228180.2,2.00,6.0,12.00,0160
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1909...,goed,147081.2,220391.8,2.45,0.0,3.35,0140


### Visualize results

Using Folium, we can display the results of our search on a map.

In [28]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

In [29]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [30]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok# initia 
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['aquifer'][loc]).add_to(marker_cluster)
fmap